<a href="https://colab.research.google.com/github/datagrad/1.ML/blob/main/Boosting_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Packages

In [1]:
import pandas as pd
import sklearn as sk
import numpy as np
import scipy as sp
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import time
from sklearn.ensemble import GradientBoostingClassifier
import xgboost
from xgboost.sklearn import XGBClassifier
from sklearn import preprocessing

# Data Importing

Dataset Background : https://www.kaggle.com/c/otto-group-product-classification-challenge/notebooks

In [2]:
ecom_prod_train=pd.read_csv("https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Ecom_Products_Menu/train.csv")
ecom_prod_test=pd.read_csv("https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Ecom_Products_Menu/test.csv")

In [8]:
ecom_prod_train

,id,spec1,spec2,spec3,spec4,spec5,spec6,spec7,spec8,spec9,...,spec92,spec93,spec94,spec95,spec96,spec97,spec98,spec99,spec100,Category
0,1,1,0,0,0,0,0,0,0,0,...,8,9,0,4,0,12,0,0,0,Mobiles
1,2,0,0,0,0,0,0,0,1,0,...,7,1,0,0,0,1,0,0,0,Mobiles
2,3,0,0,0,0,0,0,0,1,0,...,7,9,0,0,0,9,0,0,0,Mobiles
3,4,1,0,0,1,6,1,5,0,0,...,7,14,0,15,0,21,0,0,0,Mobiles
4,5,0,0,0,0,0,0,0,0,0,...,10,7,0,0,0,12,0,0,0,Mobiles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50117,61874,1,0,0,1,1,0,0,0,0,...,2,3,0,22,2,5,0,2,0,Camara
50118,61875,4,0,0,0,0,0,0,0,0,...,2,16,0,-2,0,27,0,1,0,Camara
50119,61876,0,0,0,0,0,0,0,3,1,...,2,20,0,4,0,20,0,0,0,Camara
50120,61877,1,0,0,0,0,0,0,0,0,...,3,6,0,2,0,6,3,10,0,Camara


In [3]:
print(ecom_prod_train.sample(10))
print(ecom_prod_test.sample(10))

          id  spec1  spec2  spec3  spec4  spec5  spec6  spec7  spec8  spec9  \
26852  33138      0      0      0      0      0      0      0      0      0   
9619   11846      0      0      0      0      0      0      0      0      0   
37025  45709      0      0      0      0      0      0      0      1      0   
10408  12858      0      0      0      0      0      0      0      0      0   
39994  49320      0      0      0      0      0      0      1      7      0   
21260  26233      0      0      1      0      0      0      1      0      0   
18661  23031      0      0      0      0      0      0      0      0      0   
43797  54009      1      2      0      0      0      0      2      0      0   
24041  29710      0      0      0      0      0      0      0      0      0   
13333  16456      0      0      1      0      0      0      0      2      0   

       ...  spec92  spec93  spec94  spec95  spec96  spec97  spec98  spec99  \
26852  ...       9       8       0      -2       0  

In [4]:
#Frequency counts of the products
ecom_prod_train['Category'].value_counts()

Tablets          13079
Personal_Care    11452
Appliances        6863
Laptops           6466
Camara            3964
Accessories       2341
Ipod              2217
TV                2182
Mobiles           1558
Name: Category, dtype: int64

# Prepare Features, Train and Test data

In [5]:
features=list(ecom_prod_train.columns[1:101])
print("features \n", features)
X_train=ecom_prod_train[features]
y_train=ecom_prod_train['Category']

X_test=ecom_prod_test[features]
y_test=ecom_prod_test['Category']

features 
 ['spec1', 'spec2', 'spec3', 'spec4', 'spec5', 'spec6', 'spec7', 'spec8', 'spec9', 'spec10', 'spec11', 'spec12', 'spec13', 'spec14', 'spec15', 'spec16', 'spec17', 'spec18', 'spec19', 'spec20', 'spec21', 'spec22', 'spec23', 'spec24', 'spec25', 'spec26', 'spec27', 'spec28', 'spec29', 'spec30', 'spec31', 'spec32', 'spec33', 'spec34', 'spec35', 'spec36', 'spec37', 'spec38', 'spec39', 'spec40', 'spec41', 'spec42', 'spec43', 'spec44', 'spec45', 'spec46', 'spec47', 'spec48', 'spec49', 'spec50', 'spec51', 'spec52', 'spec53', 'spec54', 'spec55', 'spec56', 'spec57', 'spec58', 'spec59', 'spec60', 'spec61', 'spec62', 'spec63', 'spec64', 'spec65', 'spec66', 'spec67', 'spec68', 'spec69', 'spec70', 'spec71', 'spec72', 'spec73', 'spec74', 'spec75', 'spec76', 'spec77', 'spec78', 'spec79', 'spec80', 'spec81', 'spec82', 'spec83', 'spec84', 'spec85', 'spec86', 'spec87', 'spec88', 'spec89', 'spec90', 'spec91', 'spec92', 'spec93', 'spec94', 'spec95', 'spec96', 'spec97', 'spec98', 'spec99', 'spec10

# Decision Tree model

In [6]:
###buildng Decision tree on the training data ####
from sklearn import tree
tree = tree.DecisionTreeClassifier(max_depth=9)
tree.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=9)

## Decision Tree Results

In [7]:
#Accuracy on train data
print("Decision Tree Results \n")
print("Accuracy on train data" , tree.score(X_train, y_train))
print("Accuracy on test data" , tree.score(X_test, y_test))

Decision Tree Results 

Accuracy on train data 0.6363672638761422
Accuracy on test data 0.6266587274583192


# Boosted Model

In [9]:
from sklearn.ensemble import GradientBoostingClassifier
boost=GradientBoostingClassifier(n_estimators=100,learning_rate=0.1, verbose=1)
##fitting the gradient boost classifier
start_time = time.time()
boost.fit(X_train,y_train)
print("Time taken by GBM "+ str((time.time() - start_time))+ " Seconds")

      Iter       Train Loss   Remaining Time 
         1           1.6679            2.58m
         2           1.5254            2.57m
         3           1.4205            2.55m
         4           1.3357            2.55m
         5           1.2661            2.52m
         6           1.2069            2.63m
         7           1.1570            2.61m
         8           1.1147            2.56m
         9           1.0746            2.52m
        10           1.0392            2.48m
        20           0.8356            2.22m
        30           0.7398            1.96m
        40           0.6852            1.67m
        50           0.6487            1.40m
        60           0.6220            1.12m
        70           0.6009           50.24s
        80           0.5838           33.61s
        90           0.5694           16.79s
       100           0.5563            0.00s
Time taken by GBM 168.61418271064758 Seconds


## GBM Results

In [10]:
###predicting Gradient boosting model on the train Data
boost_predict_train=boost.predict(X_train)
cm1 = confusion_matrix(y_train,boost_predict_train)
print(cm1)

accuracy_train=f1_score(y_train, boost_predict_train, average='micro')
print("train accuracy", accuracy_train)

[[ 1497   116    16    11   132    54   162    23   330]
 [   50  6328    91     5    24    74   174     0   117]
 [   21   196  3352     4     6    86   146     1   152]
 [    0     3     3  2144     7     0     4     0    56]
 [  124    18    11     1  2954     7     8    69  3274]
 [   32   225   271     9     5   767   149     2    98]
 [  108   188   126     1    13    50 10813    11   142]
 [   20     7     2    14   264     0    61   934   880]
 [   77    24    12    18  1377     8    32    94 11437]]
train accuracy 0.8025617493316308


In [11]:
###predicting Gradient boosting model on the test Data
boost_predict_test=boost.predict(X_test)
cm1 = confusion_matrix(y_test,boost_predict_test)
print(cm1)

accuracy_test=f1_score(y_test, boost_predict_test, average='micro')
print("test accuracy", accuracy_test)

[[ 310   31    0    1   25    9   41    5   76]
 [  14 1456   31    1    3   20   49    0   27]
 [   3   41  850    2    0   26   38    0   31]
 [   0    0    0  498    1    0    0    1   22]
 [  21    6    4    0  682    0    2   23  800]
 [   6   68   78    0    4  150   36    0   29]
 [  38   38   37    1    2   10 2512    3   42]
 [   5    1    0    3   68    0   22  169  241]
 [  20    7    7    9  329    0    8   23 2640]]
test accuracy 0.7882783259612114


# XGB Model

In [12]:
#Creating XGB Friendly data and matrices
train_labels = y_train.values
train_labels = preprocessing.LabelEncoder().fit_transform(train_labels)
test_labels = y_test.values
test_labels = preprocessing.LabelEncoder().fit_transform(test_labels)

matrix_train = xgboost.DMatrix(X_train,label=train_labels)
matrix_test = xgboost.DMatrix(X_test,label=test_labels)

In [16]:
params = {
    'max_depth': 4,
    'eta':0.1, #Learning Rate
    'eval_metric':'merror', # Multiclass classification error rate.
    'tree_method' : "gpu_hist", # use this with colab in gpu mode for faster training
    'num_class': 9
}

start_time = time.time()

model=xgboost.train(params=params,
                    dtrain=matrix_train,
                    num_boost_round=300,    #Number of trees
                    early_stopping_rounds=4, # Stop after 4 rounds, if test error doesn't improve.
                    evals=[(matrix_test,'test')]
                   )

print("Time taken by XGB "+ str((time.time() - start_time))+ " Seconds")

[0]	test-merror:0.31159
[1]	test-merror:0.29695
[2]	test-merror:0.29296
[3]	test-merror:0.28394


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:49:18] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[4]	test-merror:0.28088
[5]	test-merror:0.27518
[6]	test-merror:0.27348
[7]	test-merror:0.26956
[8]	test-merror:0.26727
[9]	test-merror:0.26735
[10]	test-merror:0.26395
[11]	test-merror:0.26267
[12]	test-merror:0.26106
[13]	test-merror:0.26114
[14]	test-merror:0.25978
[15]	test-merror:0.25851
[16]	test-merror:0.25783
[17]	test-merror:0.25519
[18]	test-merror:0.25408
[19]	test-merror:0.25272
[20]	test-merror:0.25264
[21]	test-merror:0.25187
[22]	test-merror:0.25009
[23]	test-merror:0.24855
[24]	test-merror:0.24634
[25]	test-merror:0.24702
[26]	test-merror:0.24592
[27]	test-merror:0.24600
[28]	test-merror:0.24617
[29]	test-merror:0.24592
Time taken by XGB 1.6395907402038574 Seconds


## XGB Results

In [17]:
###prediction using XGB on the train Data
boost_predict_train=model.predict(matrix_train)
cm1 = confusion_matrix(train_labels,boost_predict_train)
print(cm1)

accuracy_train=f1_score(train_labels, boost_predict_train, average='micro')
print("train accuracy", accuracy_train)

[[ 1304   187    16     7   137    40   160    28   462]
 [   47  6183   105     7    28    65   215     3   210]
 [   24   231  3165    13     5    97   198     1   230]
 [    0     1     3  2119     7     0     8     0    79]
 [  104    27    23     4  1873     5     9    37  4384]
 [   44   314   370     8     8   490   152     3   169]
 [  131   249   141     4    36    40 10624     9   218]
 [   40     4     1    20   150     0    88   620  1259]
 [   67    33    21    29   938     3    34    58 11896]]
train accuracy 0.7636167750688321


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [18:49:35] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


In [18]:
###prediction using XGB on the test Data
boost_predict_test=model.predict(matrix_test)
cm1 = confusion_matrix(test_labels,boost_predict_test)
print(cm1)

accuracy_test=f1_score(test_labels, boost_predict_test, average='micro')
print("test accuracy", accuracy_test)

[[ 270   43    3    0   30   11   40    8   93]
 [  15 1436   33    1    5   15   56    0   40]
 [   1   48  806    2    1   24   53    1   55]
 [   0    0    0  494    1    0    3    1   23]
 [  21    9    3    0  416    0    3   12 1074]
 [   3   94   87    0    2  109   32    0   44]
 [  34   45   40    0    8   11 2490    1   54]
 [   9    0    0    4   35    0   30  118  313]
 [  15   10    8   13  243    0   12   17 2725]]
test accuracy 0.7539979584892821
